# Predict The Category of the review Hackathon

In [ ]:
# Dataset Available here ---> https://machinehack.com/hackathon/uhack_sentiments_20_decode_code_words/overview

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving submission.csv to submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [ ]:
import pandas as pd
import numpy as np

## 1.Reading Data

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('submission.csv')

train.shape, test.shape, sub.shape

((6136, 14), (2631, 14), (2631, 12))

In [ ]:
train.head(2)

,Id,Review,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity
0,0,For some reason everybody complains and I'm co...,0,0,0,0,0,0,0,0,0,0,1,0
1,1,"I like everything about it, great choice of sp...",0,0,0,0,1,1,0,0,0,0,0,1


## 2.Data PreProcessing

In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import re
import itertools
def remove_noise(text):
    # Make lowercase
    text = text.apply(lambda x: " ".join(x.lower() for x in x.split()))
    
    # Remove whitespaces
    text = text.apply(lambda x: " ".join(x.strip() for x in x.split()))
    
    # Convert to string
    text = text.astype(str)
    text = text.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
        
    return text

In [ ]:
train.head()

,Id,Review,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity
0,0,For some reason everybody complains and I'm co...,0,0,0,0,0,0,0,0,0,0,1,0
1,1,"I like everything about it, great choice of sp...",0,0,0,0,1,1,0,0,0,0,0,1
2,2,Excellent ceiling fan brace. Easy to install a...,0,0,0,0,0,0,1,0,0,1,0,1
3,3,Work great easy to use . No issues at all with...,0,0,0,0,0,1,0,0,0,0,1,1
4,4,I would recommend this product because it is p...,0,0,0,0,0,0,0,0,0,1,0,1


In [ ]:
train['Review']

0       For some reason everybody complains and I'm co...
1       I like everything about it, great choice of sp...
2       Excellent ceiling fan brace. Easy to install a...
3       Work great easy to use . No issues at all with...
4       I would recommend this product because it is p...
                              ...                        
6131    Easy to install. Just be careful with the razo...
6132    Got a big pup and she had to learn about scree...
6133    We installed our Nest cam system a little over...
6134    this is a below average camera . The only thin...
6135                      So easy to install amd operate.
Name: Review, Length: 6136, dtype: object

In [ ]:
train['Review'] = remove_noise(train['Review'])
test['Review'] = remove_noise(test['Review'])

In [ ]:
train['Review']

0       reason everybody complains i'm complaining toi...
1       like everything it, great choice spray pattern...
2       excellent ceiling fan brace. easy install well...
3                work great easy use . issues hanging fan
4       would recommend product perfect watering hangi...
                              ...                        
6131    easy install. careful razor trimming excess. e...
6132    got big pup learn screen doors. fixed puncture...
6133    installed nest cam system little month ago alr...
6134    average camera . thing say decent comparison k...
6135                            easy install amd operate.
Name: Review, Length: 6136, dtype: object

## 3.Feature Engineering

In [ ]:
from nltk.tokenize import WordPunctTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tt = WordPunctTokenizer()

In [ ]:
cv = CountVectorizer(tokenizer=tt.tokenize, stop_words='english', ngram_range=(1,5))

cv.fit(train.Review.tolist() + test.Review.tolist())
train_cv = cv.transform(train.Review)
test_cv = cv.transform(test.Review)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


In [ ]:
tf_idf = TfidfVectorizer(min_df=5,  max_features=None, 
            strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
            ngram_range=(1, 5), use_idf=1.0,smooth_idf=1.0,sublinear_tf=1.0,
            stop_words = 'english')

tf_idf.fit(train.Review.tolist() + test.Review.tolist())
train_tf = tf_idf.transform(train.Review)
test_tf = tf_idf.transform(test.Review)

## 3.Modeling

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import SVC

In [ ]:
from scipy.stats.mstats import mode

In [ ]:
#Components
best_params = {'C': 0.9, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m = LogisticRegression(**best_params, class_weight='balanced')
m.fit(train_cv, train.Components)

best_params = {'alpha': 0.5}
nb = MultinomialNB(**best_params)
nb.fit(train_cv, train.Components)

best_params = {'alpha': 5e-08}
nb2 = BernoulliNB(**best_params)
nb2.fit(train_cv, train.Components)

best_params = {'C': 250, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m_tf = LogisticRegression(**best_params, class_weight='balanced')
m_tf.fit(train_tf, train.Components)

best_params = {'alpha': 0.06}
nb_tf = MultinomialNB(**best_params)
nb_tf.fit(train_tf, train.Components)

best_params = {'kernel': 'linear', 'C': 1}
svc_tf = SVC(**best_params, class_weight='balanced', probability=True)
svc_tf.fit(train_tf, train.Components)



pred1 = m.predict_proba(test_cv)
pred2 = nb.predict_proba(test_cv)
pred3 = nb2.predict_proba(test_cv)
pred4 = m_tf.predict_proba(test_tf)
pred5 = nb_tf.predict_proba(test_tf)
pred6 = svc_tf.predict_proba(test_tf)

test_pred_prob = np.mean([pred1, pred2, pred3, pred4, pred5, pred6], axis=0)
pred_Components = np.argmax(test_pred_prob, axis=1)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
pred_Components

array([1, 0, 0, ..., 0, 0, 0])

In [ ]:
train.columns = ['Id', 'Review', 'Components', 'Delivery_Customer_Support',
       'Design_Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability',
       'Polarity']

In [ ]:
#Delvery and Customer Support 
best_params = {'C': 0.9, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m = LogisticRegression(**best_params, class_weight='balanced')
m.fit(train_cv, train.Delivery_Customer_Support)

best_params = {'alpha': 0.5}
nb = MultinomialNB(**best_params)
nb.fit(train_cv, train.Delivery_Customer_Support)

best_params = {'alpha': 5e-08}
nb2 = BernoulliNB(**best_params)
nb2.fit(train_cv, train.Delivery_Customer_Support)

best_params = {'C': 250, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m_tf = LogisticRegression(**best_params, class_weight='balanced')
m_tf.fit(train_tf, train.Delivery_Customer_Support)

best_params = {'alpha': 0.06}
nb_tf = MultinomialNB(**best_params)
nb_tf.fit(train_tf, train.Delivery_Customer_Support)

best_params = {'kernel': 'linear', 'C': 1}
svc_tf = SVC(**best_params, class_weight='balanced', probability=True)
svc_tf.fit(train_tf, train.Delivery_Customer_Support)



pred1 = m.predict_proba(test_cv)
pred2 = nb.predict_proba(test_cv)
pred3 = nb2.predict_proba(test_cv)
pred4 = m_tf.predict_proba(test_tf)
pred5 = nb_tf.predict_proba(test_tf)
pred6 = svc_tf.predict_proba(test_tf)

test_pred_prob = np.mean([pred1, pred2, pred3, pred4, pred5, pred6], axis=0)
pred_Delivery_Customer_Support = np.argmax(test_pred_prob, axis=1)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
#Design 
best_params = {'C': 0.9, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m = LogisticRegression(**best_params, class_weight='balanced')
m.fit(train_cv, train.Design_Aesthetics)

best_params = {'alpha': 0.5}
nb = MultinomialNB(**best_params)
nb.fit(train_cv, train.Design_Aesthetics)

best_params = {'alpha': 5e-08}
nb2 = BernoulliNB(**best_params)
nb2.fit(train_cv, train.Design_Aesthetics)

best_params = {'C': 250, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m_tf = LogisticRegression(**best_params, class_weight='balanced')
m_tf.fit(train_tf, train.Design_Aesthetics)

best_params = {'alpha': 0.06}
nb_tf = MultinomialNB(**best_params)
nb_tf.fit(train_tf, train.Design_Aesthetics)

best_params = {'kernel': 'linear', 'C': 1}
svc_tf = SVC(**best_params, class_weight='balanced', probability=True)
svc_tf.fit(train_tf, train.Design_Aesthetics)



pred1 = m.predict_proba(test_cv)
pred2 = nb.predict_proba(test_cv)
pred3 = nb2.predict_proba(test_cv)
pred4 = m_tf.predict_proba(test_tf)
pred5 = nb_tf.predict_proba(test_tf)
pred6 = svc_tf.predict_proba(test_tf)

test_pred_prob = np.mean([pred1, pred2, pred3, pred4, pred5, pred6], axis=0)
pred_Design_Aesthetics = np.argmax(test_pred_prob, axis=1)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
#Dimension 
best_params = {'C': 0.9, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m = LogisticRegression(**best_params, class_weight='balanced')
m.fit(train_cv, train.Dimensions)

best_params = {'alpha': 0.5}
nb = MultinomialNB(**best_params)
nb.fit(train_cv, train.Dimensions)

best_params = {'alpha': 5e-08}
nb2 = BernoulliNB(**best_params)
nb2.fit(train_cv, train.Dimensions)

best_params = {'C': 250, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m_tf = LogisticRegression(**best_params, class_weight='balanced')
m_tf.fit(train_tf, train.Dimensions)

best_params = {'alpha': 0.06}
nb_tf = MultinomialNB(**best_params)
nb_tf.fit(train_tf, train.Dimensions)

best_params = {'kernel': 'linear', 'C': 1}
svc_tf = SVC(**best_params, class_weight='balanced', probability=True)
svc_tf.fit(train_tf, train.Dimensions)



pred1 = m.predict_proba(test_cv)
pred2 = nb.predict_proba(test_cv)
pred3 = nb2.predict_proba(test_cv)
pred4 = m_tf.predict_proba(test_tf)
pred5 = nb_tf.predict_proba(test_tf)
pred6 = svc_tf.predict_proba(test_tf)

test_pred_prob = np.mean([pred1, pred2, pred3, pred4, pred5, pred6], axis=0)
pred_Dimensions = np.argmax(test_pred_prob, axis=1)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
#Features
best_params = {'C': 0.9, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m = LogisticRegression(**best_params, class_weight='balanced')
m.fit(train_cv, train.Features)

best_params = {'alpha': 0.5}
nb = MultinomialNB(**best_params)
nb.fit(train_cv, train.Features)

best_params = {'alpha': 5e-08}
nb2 = BernoulliNB(**best_params)
nb2.fit(train_cv, train.Features)

best_params = {'C': 250, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m_tf = LogisticRegression(**best_params, class_weight='balanced')
m_tf.fit(train_tf, train.Features)

best_params = {'alpha': 0.06}
nb_tf = MultinomialNB(**best_params)
nb_tf.fit(train_tf, train.Features)

best_params = {'kernel': 'linear', 'C': 1}
svc_tf = SVC(**best_params, class_weight='balanced', probability=True)
svc_tf.fit(train_tf, train.Features)



pred1 = m.predict_proba(test_cv)
pred2 = nb.predict_proba(test_cv)
pred3 = nb2.predict_proba(test_cv)
pred4 = m_tf.predict_proba(test_tf)
pred5 = nb_tf.predict_proba(test_tf)
pred6 = svc_tf.predict_proba(test_tf)

test_pred_prob = np.mean([pred1, pred2, pred3, pred4, pred5, pred6], axis=0)
pred_Features = np.argmax(test_pred_prob, axis=1)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
#Functionality
best_params = {'C': 0.9, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m = LogisticRegression(**best_params, class_weight='balanced')
m.fit(train_cv, train.Functionality)

best_params = {'alpha': 0.5}
nb = MultinomialNB(**best_params)
nb.fit(train_cv, train.Functionality)

best_params = {'alpha': 5e-08}
nb2 = BernoulliNB(**best_params)
nb2.fit(train_cv, train.Functionality)

best_params = {'C': 250, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m_tf = LogisticRegression(**best_params, class_weight='balanced')
m_tf.fit(train_tf, train.Functionality)

best_params = {'alpha': 0.06}
nb_tf = MultinomialNB(**best_params)
nb_tf.fit(train_tf, train.Functionality)

best_params = {'kernel': 'linear', 'C': 1}
svc_tf = SVC(**best_params, class_weight='balanced', probability=True)
svc_tf.fit(train_tf, train.Functionality)



pred1 = m.predict_proba(test_cv)
pred2 = nb.predict_proba(test_cv)
pred3 = nb2.predict_proba(test_cv)
pred4 = m_tf.predict_proba(test_tf)
pred5 = nb_tf.predict_proba(test_tf)
pred6 = svc_tf.predict_proba(test_tf)

test_pred_prob = np.mean([pred1, pred2, pred3, pred4, pred5, pred6], axis=0)
pred_Functionality = np.argmax(test_pred_prob, axis=1)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
#installation
best_params = {'C': 0.9, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m = LogisticRegression(**best_params, class_weight='balanced')
m.fit(train_cv, train.Installation)

best_params = {'alpha': 0.5}
nb = MultinomialNB(**best_params)
nb.fit(train_cv, train.Installation)

best_params = {'alpha': 5e-08}
nb2 = BernoulliNB(**best_params)
nb2.fit(train_cv, train.Installation)

best_params = {'C': 250, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m_tf = LogisticRegression(**best_params, class_weight='balanced')
m_tf.fit(train_tf, train.Installation)

best_params = {'alpha': 0.06}
nb_tf = MultinomialNB(**best_params)
nb_tf.fit(train_tf, train.Installation)

best_params = {'kernel': 'linear', 'C': 1}
svc_tf = SVC(**best_params, class_weight='balanced', probability=True)
svc_tf.fit(train_tf, train.Installation)



pred1 = m.predict_proba(test_cv)
pred2 = nb.predict_proba(test_cv)
pred3 = nb2.predict_proba(test_cv)
pred4 = m_tf.predict_proba(test_tf)
pred5 = nb_tf.predict_proba(test_tf)
pred6 = svc_tf.predict_proba(test_tf)

test_pred_prob = np.mean([pred1, pred2, pred3, pred4, pred5, pred6], axis=0)
pred_Installation = np.argmax(test_pred_prob, axis=1)

In [ ]:
train.columns

Index(['Id', 'Review', 'Components', 'Delivery_Customer_Support',
       'Design_Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability',
       'Polarity'],
      dtype='object')

In [ ]:
#Material
best_params = {'C': 0.9, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m = LogisticRegression(**best_params, class_weight='balanced')
m.fit(train_cv, train.Material)

best_params = {'alpha': 0.5}
nb = MultinomialNB(**best_params)
nb.fit(train_cv, train.Material)

best_params = {'alpha': 5e-08}
nb2 = BernoulliNB(**best_params)
nb2.fit(train_cv, train.Material)

best_params = {'C': 250, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m_tf = LogisticRegression(**best_params, class_weight='balanced')
m_tf.fit(train_tf, train.Material)

best_params = {'alpha': 0.06}
nb_tf = MultinomialNB(**best_params)
nb_tf.fit(train_tf, train.Material)

best_params = {'kernel': 'linear', 'C': 1}
svc_tf = SVC(**best_params, class_weight='balanced', probability=True)
svc_tf.fit(train_tf, train.Material)



pred1 = m.predict_proba(test_cv)
pred2 = nb.predict_proba(test_cv)
pred3 = nb2.predict_proba(test_cv)
pred4 = m_tf.predict_proba(test_tf)
pred5 = nb_tf.predict_proba(test_tf)
pred6 = svc_tf.predict_proba(test_tf)

test_pred_prob = np.mean([pred1, pred2, pred3, pred4, pred5, pred6], axis=0)
pred_Material = np.argmax(test_pred_prob, axis=1)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
#Price
best_params = {'C': 0.9, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m = LogisticRegression(**best_params, class_weight='balanced')
m.fit(train_cv, train.Price)

best_params = {'alpha': 0.5}
nb = MultinomialNB(**best_params)
nb.fit(train_cv, train.Price)

best_params = {'alpha': 5e-08}
nb2 = BernoulliNB(**best_params)
nb2.fit(train_cv, train.Price)

best_params = {'C': 250, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m_tf = LogisticRegression(**best_params, class_weight='balanced')
m_tf.fit(train_tf, train.Price)

best_params = {'alpha': 0.06}
nb_tf = MultinomialNB(**best_params)
nb_tf.fit(train_tf, train.Price)

best_params = {'kernel': 'linear', 'C': 1}
svc_tf = SVC(**best_params, class_weight='balanced', probability=True)
svc_tf.fit(train_tf, train.Price)



pred1 = m.predict_proba(test_cv)
pred2 = nb.predict_proba(test_cv)
pred3 = nb2.predict_proba(test_cv)
pred4 = m_tf.predict_proba(test_tf)
pred5 = nb_tf.predict_proba(test_tf)
pred6 = svc_tf.predict_proba(test_tf)

test_pred_prob = np.mean([pred1, pred2, pred3, pred4, pred5, pred6], axis=0)
pred_Price = np.argmax(test_pred_prob, axis=1)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
#Quality
best_params = {'C': 0.9, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m = LogisticRegression(**best_params, class_weight='balanced')
m.fit(train_cv, train.Quality)

best_params = {'alpha': 0.5}
nb = MultinomialNB(**best_params)
nb.fit(train_cv, train.Quality)

best_params = {'alpha': 5e-08}
nb2 = BernoulliNB(**best_params)
nb2.fit(train_cv, train.Quality)

best_params = {'C': 250, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m_tf = LogisticRegression(**best_params, class_weight='balanced')
m_tf.fit(train_tf, train.Quality)

best_params = {'alpha': 0.06}
nb_tf = MultinomialNB(**best_params)
nb_tf.fit(train_tf, train.Quality)

best_params = {'kernel': 'linear', 'C': 1}
svc_tf = SVC(**best_params, class_weight='balanced', probability=True)
svc_tf.fit(train_tf, train.Quality)



pred1 = m.predict_proba(test_cv)
pred2 = nb.predict_proba(test_cv)
pred3 = nb2.predict_proba(test_cv)
pred4 = m_tf.predict_proba(test_tf)
pred5 = nb_tf.predict_proba(test_tf)
pred6 = svc_tf.predict_proba(test_tf)

test_pred_prob = np.mean([pred1, pred2, pred3, pred4, pred5, pred6], axis=0)
pred_Quality = np.argmax(test_pred_prob, axis=1)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
#Usability
best_params = {'C': 0.9, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m = LogisticRegression(**best_params, class_weight='balanced')
m.fit(train_cv, train.Usability)

best_params = {'alpha': 0.5}
nb = MultinomialNB(**best_params)
nb.fit(train_cv, train.Usability)

best_params = {'alpha': 5e-08}
nb2 = BernoulliNB(**best_params)
nb2.fit(train_cv, train.Usability)

best_params = {'C': 250, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m_tf = LogisticRegression(**best_params, class_weight='balanced')
m_tf.fit(train_tf, train.Usability)

best_params = {'alpha': 0.06}
nb_tf = MultinomialNB(**best_params)
nb_tf.fit(train_tf, train.Usability)

best_params = {'kernel': 'linear', 'C': 1}
svc_tf = SVC(**best_params, class_weight='balanced', probability=True)
svc_tf.fit(train_tf, train.Usability)



pred1 = m.predict_proba(test_cv)
pred2 = nb.predict_proba(test_cv)
pred3 = nb2.predict_proba(test_cv)
pred4 = m_tf.predict_proba(test_tf)
pred5 = nb_tf.predict_proba(test_tf)
pred6 = svc_tf.predict_proba(test_tf)

test_pred_prob = np.mean([pred1, pred2, pred3, pred4, pred5, pred6], axis=0)
pred_Usability = np.argmax(test_pred_prob, axis=1)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
#Polarity
best_params = {'C': 0.9, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m = LogisticRegression(**best_params, class_weight='balanced')
m.fit(train_cv, train.Polarity)

best_params = {'alpha': 0.5}
nb = MultinomialNB(**best_params)
nb.fit(train_cv, train.Polarity)

best_params = {'alpha': 5e-08}
nb2 = BernoulliNB(**best_params)
nb2.fit(train_cv, train.Polarity)

best_params = {'C': 250, 'max_iter': 100, 'multi_class': 'ovr', 'penalty': 'l2'}
m_tf = LogisticRegression(**best_params, class_weight='balanced')
m_tf.fit(train_tf, train.Polarity)

best_params = {'alpha': 0.06}
nb_tf = MultinomialNB(**best_params)
nb_tf.fit(train_tf, train.Polarity)

best_params = {'kernel': 'linear', 'C': 1}
svc_tf = SVC(**best_params, class_weight='balanced', probability=True)
svc_tf.fit(train_tf, train.Polarity)



pred1 = m.predict_proba(test_cv)
pred2 = nb.predict_proba(test_cv)
pred3 = nb2.predict_proba(test_cv)
pred4 = m_tf.predict_proba(test_tf)
pred5 = nb_tf.predict_proba(test_tf)
pred6 = svc_tf.predict_proba(test_tf)

test_pred_prob = np.mean([pred1, pred2, pred3, pred4, pred5, pred6], axis=0)
print(test_pred_prob)
pred_Polarity = np.argmax(test_pred_prob, axis=1)
print(pred_Polarity)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[[0.96641001 0.03358999]
 [0.15663102 0.84336898]
 [0.00843257 0.99156743]
 ...
 [0.01002148 0.98997852]
 [0.01557233 0.98442767]
 [0.00646641 0.99353359]]
[0 1 1 ... 1 1 1]


In [ ]:
train.columns

Index(['Id', 'Review', 'Components', 'Delivery_Customer_Support',
       'Design_Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability',
       'Polarity'],
      dtype='object')

## 4.Ensembing learning

In [ ]:
sub.columns

Index(['Components', 'Delivery and Customer Support', 'Design and Aesthetics',
       'Dimensions', 'Features', 'Functionality', 'Installation', 'Material',
       'Price', 'Quality', 'Usability', 'Polarity'],
      dtype='object')

In [ ]:
submission = pd.DataFrame()
submission['Components'] = pred_Components.astype(int)
submission['Delivery and Customer Support'] = pred_Delivery_Customer_Support.astype(int)
submission['Design and Aesthetics'] = pred_Design_Aesthetics.astype(int)
submission['Dimensions'] = pred_Dimensions.astype(int)
submission['Features'] = pred_Features.astype(int)
submission['Functionality'] = pred_Functionality.astype(int)
submission['Installation'] = pred_Installation.astype(int)
submission['Material'] = pred_Material.astype(int)
submission['Price'] = pred_Price.astype(int)
submission['Quality'] = pred_Quality.astype(int)
submission['Usability'] = pred_Usability.astype(int)
submission['Polarity'] = pred_Polarity.astype(int)

submission.head()

,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity
0,1,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,1,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,1,0,0,0,1,0,1
4,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
submission.to_excel("Hackathon_submission.xlsx", index=None)
